In [17]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, r2_score

# Load the dataset
file_path = 'data/bsmax_0_1.csv'
data = pd.read_csv(file_path, sep='\t')

# Drop any rows with missing values
data_cleaned = data.dropna()

# Select features and target
X = data_cleaned.drop(columns=['rtn'])
y = data_cleaned['rtn']

# Train the model on the entire dataset to get feature importances
model_full = RandomForestRegressor(n_estimators=100, random_state=42)
model_full.fit(X, y)

# Get feature importances
feature_importances = model_full.feature_importances_

# Create a DataFrame to view feature importances
feature_importances_df = pd.DataFrame({
    'Feature': X.columns,
    'Importance': feature_importances
}).sort_values(by='Importance', ascending=False)

# Select top 10 important features
top_features = feature_importances_df.head(10)['Feature'].values
X_top = data_cleaned[top_features]

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_top, y, test_size=0.2, random_state=42)

# Hyperparameter tuning for RandomForest
rf_params = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10]
}

rf_grid = GridSearchCV(RandomForestRegressor(random_state=42), rf_params, cv=5, scoring='r2', n_jobs=-1)
rf_grid.fit(X_train, y_train)

# Best RandomForest model
best_rf_model = rf_grid.best_estimator_
y_pred_rf = best_rf_model.predict(X_test)

# Evaluate the best RandomForest model
mse_rf = mean_squared_error(y_test, y_pred_rf)
r2_rf = r2_score(y_test, y_pred_rf)

# Gradient Boosting Regressor
gbr_params = {
    'n_estimators': [100, 200, 300],
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 4, 5]
}

gbr_grid = GridSearchCV(GradientBoostingRegressor(random_state=42), gbr_params, cv=5, scoring='r2', n_jobs=-1)
gbr_grid.fit(X_train, y_train)

# Best Gradient Boosting model
best_gbr_model = gbr_grid.best_estimator_
y_pred_gbr = best_gbr_model.predict(X_test)

# Evaluate the best Gradient Boosting model
mse_gbr = mean_squared_error(y_test, y_pred_gbr)
r2_gbr = r2_score(y_test, y_pred_gbr)

print("Random Forest - MSE:", mse_rf, "R2:", r2_rf)
print("Gradient Boosting - MSE:", mse_gbr, "R2:", r2_gbr)


Random Forest - MSE: 9786075.59468329 R2: 0.4509381235242692
Gradient Boosting - MSE: 9520347.897471126 R2: 0.4658471589850367


In [18]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score
from xgboost import XGBRegressor
from sklearn.preprocessing import StandardScaler

# Load the dataset
file_path = 'data/bsmax_0_1.csv'
data = pd.read_csv(file_path, sep='\t')

# Drop any rows with missing values
data_cleaned = data.dropna()

# Select features and target
X = data_cleaned.drop(columns=['rtn'])
y = data_cleaned['rtn']

# Train the model on the entire dataset to get feature importances
model_full = RandomForestRegressor(n_estimators=100, random_state=42)
model_full.fit(X, y)

# Get feature importances
feature_importances = model_full.feature_importances_

# Create a DataFrame to view feature importances
feature_importances_df = pd.DataFrame({
    'Feature': X.columns,
    'Importance': feature_importances
}).sort_values(by='Importance', ascending=False)

# Select top 10 important features
top_features = feature_importances_df.head(10)['Feature'].values
X_top = data_cleaned[top_features]

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_top, y, test_size=0.2, random_state=42)

# Standardize the data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# XGBoost Regressor
xgb_params = {
    'n_estimators': [100, 200, 300],
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 4, 5]
}

xgb_grid = GridSearchCV(XGBRegressor(random_state=42), xgb_params, cv=5, scoring='r2', n_jobs=-1)
xgb_grid.fit(X_train_scaled, y_train)

# Best XGBoost model
best_xgb_model = xgb_grid.best_estimator_
y_pred_xgb = best_xgb_model.predict(X_test_scaled)

# Evaluate the best XGBoost model
mse_xgb = mean_squared_error(y_test, y_pred_xgb)
r2_xgb = r2_score(y_test, y_pred_xgb)

print("XGBoost - MSE:", mse_xgb, "R2:", r2_xgb)


XGBoost - MSE: 9924337.897187652 R2: 0.443180717756819


In [24]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV, KFold
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, StackingRegressor
from sklearn.metrics import mean_squared_error, r2_score
from xgboost import XGBRegressor
from sklearn.preprocessing import StandardScaler

# Load the dataset
file_path = 'data/bsmax_0_1.csv'
data = pd.read_csv(file_path, sep='\t')

# Drop any rows with missing values
data_cleaned = data.dropna()

# Select features and target
X = data_cleaned.drop(columns=['rtn'])
y = data_cleaned['rtn']

# Train the model on the entire dataset to get feature importances
model_full = RandomForestRegressor(n_estimators=100, random_state=42)
model_full.fit(X, y)

# Get feature importances
feature_importances = model_full.feature_importances_

# Create a DataFrame to view feature importances
feature_importances_df = pd.DataFrame({
    'Feature': X.columns,
    'Importance': feature_importances
}).sort_values(by='Importance', ascending=False)

# Select top 10 important features
top_features = feature_importances_df.head(10)['Feature'].values
X_top = data_cleaned[top_features]

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_top, y, test_size=0.2, random_state=42)

# Standardize the data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Define base models
base_models = [
    ('rf', RandomForestRegressor(n_estimators=200, max_depth=20, min_samples_split=5, random_state=42)),
    ('gbr', GradientBoostingRegressor(n_estimators=200, learning_rate=0.1, max_depth=4, random_state=42)),
    ('xgb', XGBRegressor(n_estimators=200, learning_rate=0.1, max_depth=4, random_state=42))
]

# Define meta model
meta_model = GradientBoostingRegressor(n_estimators=200, learning_rate=0.1, max_depth=4, random_state=42)

# Create Stacking Regressor
stacking_regressor = StackingRegressor(estimators=base_models, final_estimator=meta_model, cv=KFold(n_splits=5))

# Fit the Stacking Regressor
stacking_regressor.fit(X_train_scaled, y_train)

# Predict and evaluate
y_pred_stack = stacking_regressor.predict(X_test_scaled)

# Evaluate the Stacking model
mse_stack = mean_squared_error(y_test, y_pred_stack)
r2_stack = r2_score(y_test, y_pred_stack)

print("Stacking Regressor - MSE:", mse_stack, "R2:", r2_stack)


Stacking Regressor - MSE: 10741892.101384897 R2: 0.39731066074223387


In [25]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping

# Load the dataset
data = pd.read_csv(file_path, sep='\t')

# Drop any rows with missing values
data_cleaned = data.dropna()

# Select features and target
X = data_cleaned.drop(columns=['rtn'])
y = data_cleaned['rtn']

# Train the model on the entire dataset to get feature importances
model_full = RandomForestRegressor(n_estimators=100, random_state=42)
model_full.fit(X, y)

# Get feature importances
feature_importances = model_full.feature_importances_

# Create a DataFrame to view feature importances
feature_importances_df = pd.DataFrame({
    'Feature': X.columns,
    'Importance': feature_importances
}).sort_values(by='Importance', ascending=False)

# Select top 10 important features
top_features = feature_importances_df.head(10)['Feature'].values
X_top = data_cleaned[top_features]

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_top, y, test_size=0.2, random_state=42)

# Standardize the data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Define the neural network model
model = Sequential()
model.add(Dense(64, input_dim=X_train_scaled.shape[1], activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='linear'))

# Compile the model
model.compile(optimizer='adam', loss='mse')

# Define early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Train the model
history = model.fit(X_train_scaled, y_train, validation_split=0.2, epochs=200, batch_size=32, callbacks=[early_stopping], verbose=1)

# Predict and evaluate the neural network model
y_pred_nn = model.predict(X_test_scaled)

# Evaluate the neural network model
mse_nn = mean_squared_error(y_test, y_pred_nn)
r2_nn = r2_score(y_test, y_pred_nn)

print("Neural Network - MSE:", mse_nn, "R2:", r2_nn)


Epoch 1/200


/opt/homebrew/lib/python3.11/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


46/46 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 14776562.0000 - val_loss: 17432474.0000
Epoch 2/200
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 839us/step - loss: 15307747.0000 - val_loss: 17428472.0000
Epoch 3/200
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 750us/step - loss: 13945887.0000 - val_loss: 17420086.0000
Epoch 4/200
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 742us/step - loss: 13952792.0000 - val_loss: 17402568.0000
Epoch 5/200
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 787us/step - loss: 15739523.0000 - val_loss: 17371828.0000
Epoch 6/200
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 969us/step - loss: 14235349.0000 - val_loss: 17326042.0000
Epoch 7/200
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 723us/step - loss: 13325995.0000 - val_loss: 17257180.0000
Epoch 8/200
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 722us/step - loss: 12840553.0000 - val_loss: 17159188.0000
Epoch 9/200
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 708us/step - loss: 13521615.0000 - val_loss: 17068962.0000
Epoch 10/200
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 743us/step - loss: 14746858.0000 - val_loss: 16955596.0000
Epo

In [27]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping

# Load the dataset
file_path = 'data/bsmax_0_1.csv'
data = pd.read_csv(file_path, sep='\t')

# Drop any rows with missing values
data_cleaned = data.dropna()

# Select features and target
X = data_cleaned.drop(columns=['rtn'])
y = data_cleaned['rtn']

# Train the model on the entire dataset to get feature importances
model_full = RandomForestRegressor(n_estimators=100, random_state=42)
model_full.fit(X, y)

# Get feature importances
feature_importances = model_full.feature_importances_

# Create a DataFrame to view feature importances
feature_importances_df = pd.DataFrame({
    'Feature': X.columns,
    'Importance': feature_importances
}).sort_values(by='Importance', ascending=False)

# Select top 10 important features
top_features = feature_importances_df.head(10)['Feature'].values
X_top = data_cleaned[top_features]

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_top, y, test_size=0.2, random_state=42)

# Standardize the data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Define the neural network model
model = Sequential()
model.add(Dense(128, input_dim=X_train_scaled.shape[1], activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.2))
model.add(Dense(64, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.2))
model.add(Dense(32, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.2))
model.add(Dense(1, activation='linear'))

# Compile the model
model.compile(optimizer='adam', loss='mse')

# Define early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Train the model
history = model.fit(X_train_scaled, y_train, validation_split=0.2, epochs=200, batch_size=32, callbacks=[early_stopping], verbose=1)

# Predict and evaluate the neural network model
y_pred_nn = model.predict(X_test_scaled)

# Evaluate the neural network model
mse_nn = mean_squared_error(y_test, y_pred_nn)
r2_nn = r2_score(y_test, y_pred_nn)

print("Neural Network - MSE:", mse_nn, "R2:", r2_nn)


Epoch 1/200


/opt/homebrew/lib/python3.11/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


46/46 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 14139107.0000 - val_loss: 17432540.0000
Epoch 2/200
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 13966505.0000 - val_loss: 17428142.0000
Epoch 3/200
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 13571672.0000 - val_loss: 17423936.0000
Epoch 4/200
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 14530075.0000 - val_loss: 17418338.0000
Epoch 5/200
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 15502713.0000 - val_loss: 17410858.0000
Epoch 6/200
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 14326075.0000 - val_loss: 17395424.0000
Epoch 7/200
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 14008859.0000 - val_loss: 17385546.0000
Epoch 8/200
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 14630339.0000 - val_loss: 17367032.0000
Epoch 9/200
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 15793224.0000 - val_loss: 17346382.0000
Epoch 10/200
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 14278614.0000 - val_loss: 17338828.0000
Epoch 11/200
46/46 ━━

In [28]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from xgboost import XGBRegressor
from sklearn.ensemble import StackingRegressor

# Load the dataset

data = pd.read_csv(file_path, sep='\t')

# Drop any rows with missing values
data_cleaned = data.dropna()

# Select features and target
X = data_cleaned.drop(columns=['rtn'])
y = data_cleaned['rtn']

# Train the model on the entire dataset to get feature importances
model_full = RandomForestRegressor(n_estimators=100, random_state=42)
model_full.fit(X, y)

# Get feature importances
feature_importances = model_full.feature_importances_

# Create a DataFrame to view feature importances
feature_importances_df = pd.DataFrame({
    'Feature': X.columns,
    'Importance': feature_importances
}).sort_values(by='Importance', ascending=False)

# Select top 10 important features
top_features = feature_importances_df.head(10)['Feature'].values
X_top = data_cleaned[top_features]

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_top, y, test_size=0.2, random_state=42)

# Standardize the data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Define base models
base_models = [
    ('rf', RandomForestRegressor(n_estimators=200, max_depth=20, min_samples_split=5, random_state=42)),
    ('gbr', GradientBoostingRegressor(n_estimators=200, learning_rate=0.1, max_depth=4, random_state=42)),
    ('xgb', XGBRegressor(n_estimators=200, learning_rate=0.1, max_depth=4, random_state=42))
]

# Define meta model
meta_model = GradientBoostingRegressor(n_estimators=200, learning_rate=0.1, max_depth=4, random_state=42)

# Create Stacking Regressor
stacking_regressor = StackingRegressor(estimators=base_models, final_estimator=meta_model, cv=5)

# Fit the Stacking Regressor
stacking_regressor.fit(X_train_scaled, y_train)

# Predict and evaluate
y_pred_stack = stacking_regressor.predict(X_test_scaled)

# Evaluate the Stacking model
mse_stack = mean_squared_error(y_test, y_pred_stack)
r2_stack = r2_score(y_test, y_pred_stack)

print("Stacking Regressor - MSE:", mse_stack, "R2:", r2_stack)


Stacking Regressor - MSE: 10741892.101384897 R2: 0.39731066074223387


In [30]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, Input, Add
from tensorflow.keras.callbacks import EarlyStopping

# Load the dataset
file_path = 'data/bsmax_0_1.csv'
data = pd.read_csv(file_path, sep='\t')

# Drop any rows with missing values
data_cleaned = data.dropna()

# Select features and target
X = data_cleaned.drop(columns=['rtn'])
y = data_cleaned['rtn']

# Train the model on the entire dataset to get feature importances
model_full = RandomForestRegressor(n_estimators=100, random_state=42)
model_full.fit(X, y)

# Get feature importances
feature_importances = model_full.feature_importances_

# Create a DataFrame to view feature importances
feature_importances_df = pd.DataFrame({
    'Feature': X.columns,
    'Importance': feature_importances
}).sort_values(by='Importance', ascending=False)

# Select top 10 important features
top_features = feature_importances_df.head(10)['Feature'].values
X_top = data_cleaned[top_features]

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_top, y, test_size=0.2, random_state=42)

# Standardize the data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Define early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Architecture 1: Deeper network
model_1 = Sequential()
model_1.add(Dense(128, input_dim=X_train_scaled.shape[1], activation='relu'))
model_1.add(BatchNormalization())
model_1.add(Dropout(0.2))
model_1.add(Dense(64, activation='relu'))
model_1.add(BatchNormalization())
model_1.add(Dropout(0.2))
model_1.add(Dense(32, activation='relu'))
model_1.add(BatchNormalization())
model_1.add(Dropout(0.2))
model_1.add(Dense(16, activation='relu'))
model_1.add(BatchNormalization())
model_1.add(Dropout(0.2))
model_1.add(Dense(1, activation='linear'))
model_1.compile(optimizer='adam', loss='mse')

# Train the model
history_1 = model_1.fit(X_train_scaled, y_train, validation_split=0.2, epochs=200, batch_size=32, callbacks=[early_stopping], verbose=1)

# Predict and evaluate the neural network model
y_pred_1 = model_1.predict(X_test_scaled)

# Evaluate the neural network model
mse_1 = mean_squared_error(y_test, y_pred_1)
r2_1 = r2_score(y_test, y_pred_1)

print("Deeper Neural Network - MSE:", mse_1, "R2:", r2_1)

# Architecture 2: Wider network
model_2 = Sequential()
model_2.add(Dense(256, input_dim=X_train_scaled.shape[1], activation='relu'))
model_2.add(BatchNormalization())
model_2.add(Dropout(0.2))
model_2.add(Dense(128, activation='relu'))
model_2.add(BatchNormalization())
model_2.add(Dropout(0.2))
model_2.add(Dense(64, activation='relu'))
model_2.add(BatchNormalization())
model_2.add(Dropout(0.2))
model_2.add(Dense(32, activation='relu'))
model_2.add(BatchNormalization())
model_2.add(Dropout(0.2))
model_2.add(Dense(1, activation='linear'))
model_2.compile(optimizer='adam', loss='mse')

# Train the model
history_2 = model_2.fit(X_train_scaled, y_train, validation_split=0.2, epochs=200, batch_size=32, callbacks=[early_stopping], verbose=1)

# Predict and evaluate the neural network model
y_pred_2 = model_2.predict(X_test_scaled)

# Evaluate the neural network model
mse_2 = mean_squared_error(y_test, y_pred_2)
r2_2 = r2_score(y_test, y_pred_2)

print("Wider Neural Network - MSE:", mse_2, "R2:", r2_2)

# Architecture 3: Residual connections
input_layer = Input(shape=(X_train_scaled.shape[1],))
dense_1 = Dense(128, activation='relu')(input_layer)
batch_norm_1 = BatchNormalization()(dense_1)
dropout_1 = Dropout(0.2)(batch_norm_1)
dense_2 = Dense(64, activation='relu')(dropout_1)
batch_norm_2 = BatchNormalization()(dense_2)
dropout_2 = Dropout(0.2)(batch_norm_2)
residual_1 = Add()([dropout_1, dropout_2])
dense_3 = Dense(32, activation='relu')(residual_1)
batch_norm_3 = BatchNormalization()(dense_3)
dropout_3 = Dropout(0.2)(batch_norm_3)
residual_2 = Add()([residual_1, dropout_3])
output_layer = Dense(1, activation='linear')(residual_2)

model_3 = Model(inputs=input_layer, outputs=output_layer)
model_3.compile(optimizer='adam', loss='mse')

# Train the model
history_3 = model_3.fit(X_train_scaled, y_train, validation_split=0.2, epochs=200, batch_size=32, callbacks=[early_stopping], verbose=1)

# Predict and evaluate the neural network model
y_pred_3 = model_3.predict(X_test_scaled)

# Evaluate the neural network model
mse_3 = mean_squared_error(y_test, y_pred_3)
r2_3 = r2_score(y_test, y_pred_3)

print("Residual Neural Network - MSE:", mse_3, "R2:", r2_3)


Epoch 1/200


/opt/homebrew/lib/python3.11/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


46/46 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 14201680.0000 - val_loss: 17434498.0000
Epoch 2/200
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 13660483.0000 - val_loss: 17433066.0000
Epoch 3/200
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 14278575.0000 - val_loss: 17430938.0000
Epoch 4/200
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 13229753.0000 - val_loss: 17427526.0000
Epoch 5/200
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 13386334.0000 - val_loss: 17423102.0000
Epoch 6/200
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 13957774.0000 - val_loss: 17417218.0000
Epoch 7/200
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 15578202.0000 - val_loss: 17411372.0000
Epoch 8/200
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 13675641.0000 - val_loss: 17406488.0000
Epoch 9/200
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 14427964.0000 - val_loss: 17399188.0000
Epoch 10/200
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 13752070.0000 - val_loss: 17392882.0000
Epoch 11/200
46/46 ━━

/opt/homebrew/lib/python3.11/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


46/46 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 14344789.0000 - val_loss: 17433370.0000
Epoch 2/200
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 14679839.0000 - val_loss: 17429294.0000
Epoch 3/200
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 14904219.0000 - val_loss: 17423314.0000
Epoch 4/200
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 15707911.0000 - val_loss: 17416118.0000
Epoch 5/200
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 14670754.0000 - val_loss: 17402470.0000
Epoch 6/200
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 14258676.0000 - val_loss: 17390124.0000
Epoch 7/200
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 14788615.0000 - val_loss: 17375880.0000
Epoch 8/200
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 14094423.0000 - val_loss: 17356318.0000
Epoch 9/200
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 14817241.0000 - val_loss: 17343954.0000
Epoch 10/200
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 15001070.0000 - val_loss: 17320502.0000
15/15 ━━━━━━━━━━━━━━━

ValueError: Inputs have incompatible shapes. Received shapes (128,) and (64,)